In [1]:
#import tensorflow as tf
import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
import numba as nb
import sys

In [2]:
import cProfile

In [3]:
# entradas
# numero de vaos em x
vX = 3
# distancia dos vaos em x
dX = 5.25
# número de vãos em y
vY = 5
# distancia dos vaos em y
dY = 7.25
# numero pavimentos
vZ = 7
 # distancia piso a piso
dZ = 3.25

In [4]:
Npts = (vX*2 +1) * (vY*2 +1) * (vZ) + ( (vX +1) * (vY +1) )
mat = np.arange(0,Npts, 1, dtype=np.int64 )

In [34]:
@nb.vectorize(["float64(int64,int64,int64,float64)"], target='parallel')
def coordX(M, vx, vy, dx):
    if M <  ( (vx+1)* (vy+1) ):
        return  (M % (vx+1) ) * (dx)
    else:
        return  ( ( M - (vx+1) * (vy+1) ) % (vx*2 +1) ) * (dx/2) 

In [35]:
@nb.vectorize(["float64(int64,int64,int64,float64)"], target='parallel')
def coordY(M, vx, vy, dy):
    if M <  ( (vx+1)* (vy+1) ):
        return  ((M % ( (vx+1) * (vy+1) )// (vx+1) ) ) * (dy)
    else:
        return  ( ((M - (vx+1) * (vy+1) ) % ( (vx*2 +1) * (vy*2 +1) ))// (vx*2 +1) ) * (dy/2)

In [36]:
@nb.vectorize(["float64(int64,int64,int64,float64)"], target='parallel')
def coordZ(M, vx, vy, dz):
    if M < ( (vx+1)* (vy+1) ):
        return  0.0
    else:
        return  (((M - ( (vx+1)* (vy+1) ) ) // ( (vx*2 +1)*(vy*2 +1))) + 1)*(dZ)

In [5]:
@nb.jit
def estruct(vx, dx, vy, dy, vz, dz):
    # numero total de pontos
    Npts = (vx*2 +1) * (vy*2 +1) * (vz) + ( (vx +1) * (vy +1) )
    mat = np.arange(0,Npts, 1, dtype=np.int64 )
    cX = coordX(mat, vx, vy, dx)
    cY = coordY(mat, vx, vy, dy)
    cZ = coordZ(mat, vx, vy, dz)
    

In [37]:

print( '\ncoord X\n', cX , '\nCoord Y\n', cY, '\nCoord Z\n', cZ)


coord X
 [ 0.     5.25  10.5   15.75   0.     5.25  10.5   15.75   0.     5.25
 10.5   15.75   0.     5.25  10.5   15.75   0.     5.25  10.5   15.75
  0.     5.25  10.5   15.75   0.     2.625  5.25   7.875 10.5   13.125
 15.75   0.     2.625  5.25   7.875 10.5   13.125 15.75   0.     2.625
  5.25   7.875 10.5   13.125 15.75   0.     2.625  5.25   7.875 10.5
 13.125 15.75   0.     2.625  5.25   7.875 10.5   13.125 15.75   0.
  2.625  5.25   7.875 10.5   13.125 15.75   0.     2.625  5.25   7.875
 10.5   13.125 15.75   0.     2.625  5.25   7.875 10.5   13.125 15.75
  0.     2.625  5.25   7.875 10.5   13.125 15.75   0.     2.625  5.25
  7.875 10.5   13.125 15.75   0.     2.625  5.25   7.875 10.5   13.125
 15.75   0.     2.625  5.25   7.875 10.5   13.125 15.75   0.     2.625
  5.25   7.875 10.5   13.125 15.75   0.     2.625  5.25   7.875 10.5
 13.125 15.75   0.     2.625  5.25   7.875 10.5   13.125 15.75   0.
  2.625  5.25   7.875 10.5   13.125 15.75   0.     2.625  5.25   7.875
 10.5   13

In [38]:
matA = mat[(vX+1)*(vY+1)::].reshape(vZ, vY*2 +1, vX*2 +1)

In [39]:
matXI = matA[::,::2,:-1:]
matXI

array([[[ 24,  25,  26,  27,  28,  29],
        [ 38,  39,  40,  41,  42,  43],
        [ 52,  53,  54,  55,  56,  57],
        [ 66,  67,  68,  69,  70,  71],
        [ 80,  81,  82,  83,  84,  85],
        [ 94,  95,  96,  97,  98,  99]],

       [[101, 102, 103, 104, 105, 106],
        [115, 116, 117, 118, 119, 120],
        [129, 130, 131, 132, 133, 134],
        [143, 144, 145, 146, 147, 148],
        [157, 158, 159, 160, 161, 162],
        [171, 172, 173, 174, 175, 176]],

       [[178, 179, 180, 181, 182, 183],
        [192, 193, 194, 195, 196, 197],
        [206, 207, 208, 209, 210, 211],
        [220, 221, 222, 223, 224, 225],
        [234, 235, 236, 237, 238, 239],
        [248, 249, 250, 251, 252, 253]],

       [[255, 256, 257, 258, 259, 260],
        [269, 270, 271, 272, 273, 274],
        [283, 284, 285, 286, 287, 288],
        [297, 298, 299, 300, 301, 302],
        [311, 312, 313, 314, 315, 316],
        [325, 326, 327, 328, 329, 330]],

       [[332, 333, 334, 335, 336

In [40]:
matXF = matA[::,::2,1::]
matXF

array([[[ 25,  26,  27,  28,  29,  30],
        [ 39,  40,  41,  42,  43,  44],
        [ 53,  54,  55,  56,  57,  58],
        [ 67,  68,  69,  70,  71,  72],
        [ 81,  82,  83,  84,  85,  86],
        [ 95,  96,  97,  98,  99, 100]],

       [[102, 103, 104, 105, 106, 107],
        [116, 117, 118, 119, 120, 121],
        [130, 131, 132, 133, 134, 135],
        [144, 145, 146, 147, 148, 149],
        [158, 159, 160, 161, 162, 163],
        [172, 173, 174, 175, 176, 177]],

       [[179, 180, 181, 182, 183, 184],
        [193, 194, 195, 196, 197, 198],
        [207, 208, 209, 210, 211, 212],
        [221, 222, 223, 224, 225, 226],
        [235, 236, 237, 238, 239, 240],
        [249, 250, 251, 252, 253, 254]],

       [[256, 257, 258, 259, 260, 261],
        [270, 271, 272, 273, 274, 275],
        [284, 285, 286, 287, 288, 289],
        [298, 299, 300, 301, 302, 303],
        [312, 313, 314, 315, 316, 317],
        [326, 327, 328, 329, 330, 331]],

       [[333, 334, 335, 336, 337

In [41]:
matYI = matA[::,:-1:,::2]
matYI

array([[[ 24,  26,  28,  30],
        [ 31,  33,  35,  37],
        [ 38,  40,  42,  44],
        [ 45,  47,  49,  51],
        [ 52,  54,  56,  58],
        [ 59,  61,  63,  65],
        [ 66,  68,  70,  72],
        [ 73,  75,  77,  79],
        [ 80,  82,  84,  86],
        [ 87,  89,  91,  93]],

       [[101, 103, 105, 107],
        [108, 110, 112, 114],
        [115, 117, 119, 121],
        [122, 124, 126, 128],
        [129, 131, 133, 135],
        [136, 138, 140, 142],
        [143, 145, 147, 149],
        [150, 152, 154, 156],
        [157, 159, 161, 163],
        [164, 166, 168, 170]],

       [[178, 180, 182, 184],
        [185, 187, 189, 191],
        [192, 194, 196, 198],
        [199, 201, 203, 205],
        [206, 208, 210, 212],
        [213, 215, 217, 219],
        [220, 222, 224, 226],
        [227, 229, 231, 233],
        [234, 236, 238, 240],
        [241, 243, 245, 247]],

       [[255, 257, 259, 261],
        [262, 264, 266, 268],
        [269, 271, 273, 275],
    

In [42]:
matYF = matA[::,1::,::2]
matYF

array([[[ 31,  33,  35,  37],
        [ 38,  40,  42,  44],
        [ 45,  47,  49,  51],
        [ 52,  54,  56,  58],
        [ 59,  61,  63,  65],
        [ 66,  68,  70,  72],
        [ 73,  75,  77,  79],
        [ 80,  82,  84,  86],
        [ 87,  89,  91,  93],
        [ 94,  96,  98, 100]],

       [[108, 110, 112, 114],
        [115, 117, 119, 121],
        [122, 124, 126, 128],
        [129, 131, 133, 135],
        [136, 138, 140, 142],
        [143, 145, 147, 149],
        [150, 152, 154, 156],
        [157, 159, 161, 163],
        [164, 166, 168, 170],
        [171, 173, 175, 177]],

       [[185, 187, 189, 191],
        [192, 194, 196, 198],
        [199, 201, 203, 205],
        [206, 208, 210, 212],
        [213, 215, 217, 219],
        [220, 222, 224, 226],
        [227, 229, 231, 233],
        [234, 236, 238, 240],
        [241, 243, 245, 247],
        [248, 250, 252, 254]],

       [[262, 264, 266, 268],
        [269, 271, 273, 275],
        [276, 278, 280, 282],
    

In [43]:
matPF = matA[::,::2,::2]
matPF

array([[[ 24,  26,  28,  30],
        [ 38,  40,  42,  44],
        [ 52,  54,  56,  58],
        [ 66,  68,  70,  72],
        [ 80,  82,  84,  86],
        [ 94,  96,  98, 100]],

       [[101, 103, 105, 107],
        [115, 117, 119, 121],
        [129, 131, 133, 135],
        [143, 145, 147, 149],
        [157, 159, 161, 163],
        [171, 173, 175, 177]],

       [[178, 180, 182, 184],
        [192, 194, 196, 198],
        [206, 208, 210, 212],
        [220, 222, 224, 226],
        [234, 236, 238, 240],
        [248, 250, 252, 254]],

       [[255, 257, 259, 261],
        [269, 271, 273, 275],
        [283, 285, 287, 289],
        [297, 299, 301, 303],
        [311, 313, 315, 317],
        [325, 327, 329, 331]],

       [[332, 334, 336, 338],
        [346, 348, 350, 352],
        [360, 362, 364, 366],
        [374, 376, 378, 380],
        [388, 390, 392, 394],
        [402, 404, 406, 408]],

       [[409, 411, 413, 415],
        [423, 425, 427, 429],
        [437, 439, 441, 443],


In [44]:
matB = mat[: (vX+1)* (vY+1)]

In [45]:
matPI = np.append(matB,matPF[:-1:,::,::]).reshape(vZ,(vY+1),(vX+1))
matPI 

array([[[  0,   1,   2,   3],
        [  4,   5,   6,   7],
        [  8,   9,  10,  11],
        [ 12,  13,  14,  15],
        [ 16,  17,  18,  19],
        [ 20,  21,  22,  23]],

       [[ 24,  26,  28,  30],
        [ 38,  40,  42,  44],
        [ 52,  54,  56,  58],
        [ 66,  68,  70,  72],
        [ 80,  82,  84,  86],
        [ 94,  96,  98, 100]],

       [[101, 103, 105, 107],
        [115, 117, 119, 121],
        [129, 131, 133, 135],
        [143, 145, 147, 149],
        [157, 159, 161, 163],
        [171, 173, 175, 177]],

       [[178, 180, 182, 184],
        [192, 194, 196, 198],
        [206, 208, 210, 212],
        [220, 222, 224, 226],
        [234, 236, 238, 240],
        [248, 250, 252, 254]],

       [[255, 257, 259, 261],
        [269, 271, 273, 275],
        [283, 285, 287, 289],
        [297, 299, 301, 303],
        [311, 313, 315, 317],
        [325, 327, 329, 331]],

       [[332, 334, 336, 338],
        [346, 348, 350, 352],
        [360, 362, 364, 366],
